# ComfyUI FLUX Setup for Google Colab
This notebook adapts the `flux_install.sh` script for use in Google Colab to set up ComfyUI with FLUX-specific nodes and dependencies.

## 1. Configuration

## 1.1. Mount Google Drive (Optional)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# You can then access your Drive files at /content/drive/MyDrive/

In [ ]:
# --- Configuration ---
COMFYUI_DIR = "/content/ComfyUI"  # Main directory for ComfyUI installation in Colab
UPDATE_COMFY_UI = True       # Set to False to disable ComfyUI git pull if already cloned
INSTALL_CUSTOM_NODES_DEPENDENCIES = True
USE_COMFYUI_MANAGER = True
INSTALL_SAGEATTENTION = "background" # "true", "false", or "background" (background not directly applicable in Colab, will be 'true')

print(f"Target Directory: {COMFYUI_DIR}")
print(f"Update ComfyUI: {UPDATE_COMFY_UI}")
print(f"Install Custom Nodes & Dependencies: {INSTALL_CUSTOM_NODES_DEPENDENCIES}")
print(f"Use ComfyUI Manager: {USE_COMFYUI_MANAGER}")
print(f"SageAttention Installation: {INSTALL_SAGEATTENTION}")

## 2. Create Directories and Clone ComfyUI

In [ ]:
import os

# Create parent directory for COMFYUI_DIR if it doesn't exist
if not os.path.exists(os.path.dirname(COMFYUI_DIR)):
    os.makedirs(os.path.dirname(COMFYUI_DIR))

os.chdir(os.path.dirname(COMFYUI_DIR)) # Navigate to parent directory

if not os.path.exists(COMFYUI_DIR):
    print("--- Cloning ComfyUI ---")
    !git clone https://github.com/comfyanonymous/ComfyUI.git "{COMFYUI_DIR}"
    os.chdir(COMFYUI_DIR)
else:
    os.chdir(COMFYUI_DIR)
    if UPDATE_COMFY_UI:
        print("--- Updating ComfyUI ---")
        !git pull
    else:
        print("--- ComfyUI exists, skipping update as per configuration. ---")

## 3. Install ComfyUI Main Dependencies

In [ ]:
print("--- Installing ComfyUI main dependencies (from requirements.txt) ---")
%pip install -r requirements.txt

## 4. Install Additional Core Dependencies

In [ ]:
print("--- Installing additional core dependencies ---")
%pip install accelerate einops transformers>=4.28.1 safetensors>=0.4.2 aiohttp pyyaml Pillow scipy tqdm psutil tokenizers>=0.13.3
%pip install torchsde
%pip install kornia>=0.7.1 spandrel soundfile sentencepiece
%pip install imageio-ffmpeg  # For VHS_VideoCombine node

## 5. Install prerequisites for ComfyUI-Manager CLI

In [ ]:
print("--- Installing GitPython and Typer (for ComfyUI-Manager CLI) ---")
%pip install GitPython typer

## 6. Setup ComfyUI-Manager

In [ ]:
if USE_COMFYUI_MANAGER:
    print("--- Setting up ComfyUI-Manager ---")
    MANAGER_DIR = os.path.join(COMFYUI_DIR, "custom_nodes/ComfyUI-Manager")
    if not os.path.exists(MANAGER_DIR):
        print("Cloning ComfyUI-Manager...")
        !git clone https://github.com/ltdrdata/ComfyUI-Manager.git "{MANAGER_DIR}"
    else:
        print("Updating ComfyUI-Manager...")
        os.chdir(MANAGER_DIR)
        !git pull
        os.chdir(COMFYUI_DIR) # Return to ComfyUI base

    # Install requirements for ComfyUI-Manager
    manager_req_file = os.path.join(MANAGER_DIR, "requirements.txt")
    if os.path.exists(manager_req_file):
        print("Installing ComfyUI-Manager's own requirements...")
        %pip install -r "{manager_req_file}"

    print("Attempting to restore dependencies via ComfyUI-Manager CLI...")
    # Note: The cm-cli.py script might need adjustments or specific handling in Colab
    # For now, we call it directly. Ensure it's executable if needed.
    !python "{os.path.join(MANAGER_DIR, 'cm-cli.py')}" restore-dependencies

## 7. Install Custom Nodes and Their Dependencies

In [ ]:
if INSTALL_CUSTOM_NODES_DEPENDENCIES:
    print("--- Installing Custom Nodes & Their Dependencies ---")
    CUSTOM_NODES_DIR = os.path.join(COMFYUI_DIR, "custom_nodes")
    if not os.path.exists(CUSTOM_NODES_DIR):
        os.makedirs(CUSTOM_NODES_DIR)
    os.chdir(CUSTOM_NODES_DIR)

    CUSTOM_NODES_REPOS = [
        "https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite",
        "https://github.com/civitai/civitai_comfy_nodes",
        "https://github.com/kijai/ComfyUI-KJNodes",
        "https://github.com/cubiq/ComfyUI_essentials",
        "https://github.com/sipherxyz/comfyui-art-venture",
        "https://github.com/twri/sdxl_prompt_styler",
        "https://github.com/Nourepide/ComfyUI-Allor",
        "https://github.com/Extraltodeus/sigmas_tools_and_the_golden_scheduler",
        "https://github.com/rgthree/rgthree-comfy",
        "https://github.com/pollockjj/ComfyUI-MultiGPU",
        "https://github.com/daxcay/ComfyUI-JDCN",
        "https://github.com/city96/ComfyUI-GGUF",
        "https://github.com/calcuis/gguf",
        "https://github.com/kijai/ComfyUI-GIMM-VFI"
    ]

    for repo_url in CUSTOM_NODES_REPOS:
        repo_name = repo_url.split('/')[-1].replace('.git', '')
        if not os.path.exists(repo_name):
            print(f"Cloning {repo_name}...")
            !git clone {repo_url} {repo_name}
        else:
            print(f"Updating {repo_name}...")
            os.chdir(repo_name)
            !git pull
            os.chdir(CUSTOM_NODES_DIR) # Go back to custom_nodes directory

    # Install requirements for specific nodes
    NODES_WITH_REQS = ["ComfyUI-GGUF", "ComfyUI-JDCN", "ComfyUI-KJNodes"]
    for node_name in NODES_WITH_REQS:
        node_req_file = os.path.join(CUSTOM_NODES_DIR, node_name, "requirements.txt")
        if os.path.exists(node_req_file):
            print(f"Installing dependencies for {node_name}...")
            %pip install -r "{node_req_file}"

    # Special handling for was-node-suite-comfyui
    WAS_NODE_DIR_NAME = "was-node-suite-comfyui"
    WAS_NODE_REPO = "https://github.com/WASasquatch/was-node-suite-comfyui.git"
    if not os.path.exists(WAS_NODE_DIR_NAME):
        print(f"Cloning {WAS_NODE_DIR_NAME}...")
        !git clone {WAS_NODE_REPO} {WAS_NODE_DIR_NAME}
    else:
        print(f"Updating {WAS_NODE_DIR_NAME}...")
        os.chdir(WAS_NODE_DIR_NAME)
        !git pull
        os.chdir(CUSTOM_NODES_DIR) # Go back to custom_nodes directory
    
    was_req_file = os.path.join(CUSTOM_NODES_DIR, WAS_NODE_DIR_NAME, "requirements.txt")
    if os.path.exists(was_req_file):
        print(f"Installing dependencies for {WAS_NODE_DIR_NAME}...")
        %pip install -r "{was_req_file}"

    os.chdir(COMFYUI_DIR) # Return to the main ComfyUI directory

## 8. Clone FFmpeg (Optional - for building from source, usually not needed in Colab)

In [ ]:
print("--- Cloning FFmpeg source (optional) ---")
os.chdir(COMFYUI_DIR)
if not os.path.exists("ffmpeg"):
    print("Cloning FFmpeg repository...")
    !git clone https://git.ffmpeg.org/ffmpeg.git ffmpeg # Clone into a directory named ffmpeg
else:
    print("FFmpeg directory already exists. Skipping clone.")

## 9. Install gguf-node

In [ ]:
print("--- Installing gguf-node ---")
%pip install gguf-node

## 10. Install SageAttention

In [ ]:
# In Colab, 'background' installation isn't directly applicable like in a shell script.
# We'll treat 'background' as 'true'.
if INSTALL_SAGEATTENTION != "false":
    print(f"--- Setting up SageAttention (mode: {INSTALL_SAGEATTENTION}) ---")
    os.chdir(COMFYUI_DIR) # Ensure we are in the ComfyUI directory
    SAGE_DIR = os.path.join(COMFYUI_DIR, "SageAttention")
    if not os.path.exists(SAGE_DIR):
        print("Cloning SageAttention repository...")
        !git clone https://github.com/thu-ml/SageAttention.git SageAttention
    else:
        print("SageAttention directory already exists. Updating...")
        os.chdir(SAGE_DIR)
        !git pull
        os.chdir(COMFYUI_DIR) # Return to ComfyUI base

    # Install SageAttention
    # The original script had a complex backgrounding logic here.
    # For Colab, we'll do a direct install.
    print("Installing SageAttention... (This may take a while)")
    os.chdir(SAGE_DIR) # Navigate into SageAttention directory
    !python setup.py install
    os.chdir(COMFYUI_DIR) # Return to ComfyUI base
    print("SageAttention installation attempted.")
else:
    print("--- Skipping SageAttention installation as per configuration. ---")

## 11. Download Models (Placeholder)

In [ ]:
# This section corresponds to model_downloader.py
print("--- Model Downloading ---")

# 1. Define your GitHub repository and the path to the files
YOUR_REPO_URL = "https://github.com/bbits10/comfy_template.git"
# If model_downloader.py and model_configs.json are in a subdirectory of your repo, adjust this path:
PATH_TO_FILES_IN_REPO = "" # e.g., "scripts/" or "" if they are in the root

# 2. Clone your repository
print(f"Cloning repository: {YOUR_REPO_URL}")
REPO_NAME = YOUR_REPO_URL.split('/')[-1].replace('.git', '')
CLONED_REPO_PATH = f"/content/{REPO_NAME}"

# Check if the repository directory already exists
if os.path.exists(CLONED_REPO_PATH):
    print(f"Repository already cloned at {CLONED_REPO_PATH}. Skipping clone.")
else:
    !git clone {YOUR_REPO_URL} {CLONED_REPO_PATH}

# 3. Copy model_downloader.py and model_configs.json to COMFYUI_DIR
print(f"Copying model downloader script and config to {COMFYUI_DIR}")
MODEL_DOWNLOADER_SCRIPT_SOURCE = os.path.join(CLONED_REPO_PATH, PATH_TO_FILES_IN_REPO, "model_downloader.py")
MODEL_CONFIGS_JSON_SOURCE = os.path.join(CLONED_REPO_PATH, PATH_TO_FILES_IN_REPO, "model_configs.json")

MODEL_DOWNLOADER_SCRIPT_DEST = os.path.join(COMFYUI_DIR, "model_downloader.py")
MODEL_CONFIGS_JSON_DEST = os.path.join(COMFYUI_DIR, "model_configs.json")

if os.path.exists(MODEL_DOWNLOADER_SCRIPT_SOURCE):
    !cp -f {MODEL_DOWNLOADER_SCRIPT_SOURCE} {MODEL_DOWNLOADER_SCRIPT_DEST} # Added -f to overwrite if exists
    print(f"Copied model_downloader.py to {MODEL_DOWNLOADER_SCRIPT_DEST}")
else:
    print(f"ERROR: model_downloader.py not found at {MODEL_DOWNLOADER_SCRIPT_SOURCE}")

if os.path.exists(MODEL_CONFIGS_JSON_SOURCE):
    !cp -f {MODEL_CONFIGS_JSON_SOURCE} {MODEL_CONFIGS_JSON_DEST} # Added -f to overwrite if exists
    print(f"Copied model_configs.json to {MODEL_CONFIGS_JSON_DEST}")
else:
    print(f"ERROR: model_configs.json not found at {MODEL_CONFIGS_JSON_SOURCE}")

# 4. Run the model downloader script
# Ensure model_downloader.py is executable if needed: !chmod +x {MODEL_DOWNLOADER_SCRIPT_DEST}
print("Running model_downloader.py...")
os.chdir(COMFYUI_DIR) # Ensure we are in the ComfyUI directory

# Check if the script and config exist before running
if os.path.exists(MODEL_DOWNLOADER_SCRIPT_DEST) and os.path.exists(MODEL_CONFIGS_JSON_DEST):
    # Assuming model_downloader.py looks for model_configs.json in its current directory or a predefined path
    # If it takes the config path as an argument, modify the command below, e.g.:
    # !python model_downloader.py --config model_configs.json
    !python model_downloader.py
    print("Model downloader script executed.")
else:
    print("Skipping model_downloader.py execution due to missing files.")

# Fallback or direct download example (if model_downloader.py fails or is not used)
# print("--- Example: Downloading a specific model manually ---")
# SDXL_MODELS_DIR = os.path.join(COMFYUI_DIR, 'models', 'checkpoints')
# if not os.path.exists(SDXL_MODELS_DIR):
#     os.makedirs(SDXL_MODELS_DIR)
# !wget -P {SDXL_MODELS_DIR} https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors
# print("Manual model download example complete.")

## 12. Final Steps & Starting ComfyUI

In [ ]:
print("--- Setup Complete ---")
print(f"ComfyUI should be installed in: {COMFYUI_DIR}")
print("Starting ComfyUI with cloudflared tunnel...")

# 1. Download and Install cloudflared
print("\n--- Installing cloudflared ---")
!wget -q -P /tmp https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i /tmp/cloudflared-linux-amd64.deb

# 2. Define function to start cloudflared tunnel
import subprocess
import threading
import time
import socket
import urllib.request
import os

comfyui_port = 8188 # Default ComfyUI port

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared...")
  print("(If it gets stuck here, cloudflared might be having issues or ComfyUI didn't start correctly.)\n")

  # Start cloudflared tunnel
  p = subprocess.Popen(["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com" in l:
      print(f"\n!!! THIS IS YOUR PUBLIC URL TO ACCESS COMFYUI !!!\n{l[l.find('https'):].strip()}\n")
    # You can uncomment the next line to see all cloudflared output for debugging
    # print(l, end='')

# 3. Start the ComfyUI server and the cloudflared tunnel
os.chdir(COMFYUI_DIR) # Navigate to ComfyUI directory

# Start cloudflared in a background thread
threading.Thread(target=iframe_thread, daemon=True, args=(comfyui_port,)).start()

print(f"\n--- Starting ComfyUI on port {comfyui_port} ---")
# Start ComfyUI. --dont-print-server is used as cloudflared provides the URL.
# Use --preview-method auto for better preview generation in Colab if needed.
!python main.py --listen --port {comfyui_port} --dont-print-server --preview-method auto